In [10]:
import numpy as np
import matplotlib.pyplot as plt

In [11]:
def FNS(scores):
    domination = (scores[:, None, :] <= scores[None, :, :]).all(2) # domination [i, j] = "i dominuje j"
    domination &= ~(scores[:, None, :] == scores[None, :, :]).all(2)
    Nx = domination.sum(0)
    
    Pf = []
    ranks = np.zeros(scores.shape[0])
    r = 0
    Q = np.argwhere(Nx == 0)
    while Q.size > 0:
        Q = Q[:, 0]
        Nx[Q] = -1
        Pf.append(Q)
        ranks[Q] = r
        r += 1
        for i in Q:
            Nx[domination[i, :]] -= 1
        Q = np.argwhere(Nx == 0)
        
    return Pf, ranks

def crowding_distance(scores):
    indices = np.argsort(scores, 0)
    sorted_scores = np.take_along_axis(scores, indices, 0)
    cd = np.zeros(scores.shape[0])
    for k in range(scores.shape[1]):
        cd[indices[[0, -1], k]] = np.inf
        cd[indices[1:-1, k]] += (sorted_scores[2:, k] - sorted_scores[:-2, k]) / (sorted_scores[-1, k] - sorted_scores[0, k])
    return cd

def random_population(d, n, x_min, x_max):
    return np.hstack([np.random.uniform(x_min, x_max, (n, d))])

def tournament_selection(ranks, dists, n):
    candidates = np.random.choice(n, (n, 2), replace=True)
    mask = np.where(
        ranks[candidates[:, 0]] == ranks[candidates[:, 1]],
        dists[candidates[:, 0]] > dists[candidates[:, 1]],
        ranks[candidates[:, 0]] < ranks[candidates[:, 1]]
    )
    result = candidates[:, 1]
    result[mask] = candidates[mask, 0]
    return result

def crossover(x, p, eta): # simulated binary crossover
    n, d = x.shape
    l = n // 2
    mask = np.random.random(l) <= p
    m = np.sum(mask)
    mi = np.random.random((m, d))
    beta = np.where(
        mi < 0.5,
        np.power(2*mi, 1. / (eta+1.)),
        np.power(1. / (2.*(1-mi)), 1. / (eta+1.))
    )
    c1 = x[:l, :].copy()
    c2 = x[l:, :].copy()
    c1[mask, :] = 0.5 * (1 + beta) * x[:l, :][mask, :] + 0.5 * (1 - beta) * x[l:, :][mask, :]
    c2[mask, :] = 0.5 * (1 + beta) * x[:l, :][mask, :] + 0.5 * (1 - beta) * x[l:, :][mask, :]
    return np.vstack([c1, c2])

def mutation(x, x_min, x_max, p, eta): # polynomial mutation
    n, d = x.shape
    mask = np.random.random(x.shape[0]) <= p
    m = np.sum(mask)
    mi = np.random.random((m, d))
    beta = np.where(
        mi < 0.5,
        np.power(2*mi, 1. / (eta+1.)) - 1.,
        1. - np.power(2.*(1-mi), 1. / (eta+1.))
    )
    y = x.copy()
    y[mask, :] = np.where(
        mi < 0.5,
        x[mask, :] + beta * (x[mask, :] - x_min),
        x[mask, :] + beta * (x_max - x[mask, :])
    )
    return y

def elitist_selection(fronts, dists, to_take):
    taken = []
    for front in fronts:
        if len(front) <= to_take:
            taken += list(front)
            if len(front) == to_take:
                break
            to_take -= len(front)
        else:
            indices = np.argsort(-dists[front])[:to_take]
            taken += list(indices)
            break
    return taken

def constraint_violation(constraints):
    n, d = constraints.shape
    sort_indices = np.argsort(constraints, 0)
    violations = np.zeros(n)
    for i in range(d):
        values, counts = np.unique(constraints[:, i], return_counts=True)
        counts = np.cumsum(counts)
        counts = list(counts)
        if values[0] != 0:
            counts = [0] + counts
        for rank, (j, k) in enumerate(zip([0] + counts, counts + [len(counts)])):
            violations[sort_indices[j:k, i]] += rank
    return violations
        

def IDEA(objective, constraint, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=10): # TODO
    n_f = n - n_inf
    population = random_population(d, n, x_min, x_max)
    populations = [population.copy()]
    constraint_values = constraint(population)
    violation_measure = constraint_violation(constraint_values)
    scores = np.stack([objective(population), violation_measure], 1)
    scores_hist = [scores.copy()]

    fronts, ranks = FNS(scores)
    dists = crowding_distance(scores)
    
    for iter_ in range(num_iterations):
        parent_indices = tournament_selection(ranks, dists, n)
        offspring = crossover(population[parent_indices, :], p_c, eta_c)
        offspring = np.clip(offspring, x_min, x_max)
        offspring = mutation(offspring, x_min, x_max, p_m, eta_m)
        population = np.vstack([population, offspring])
        
        constraint_values = constraint(population)
        violation_measure = constraint_violation(constraint_values)
        scores = np.stack([objective(population), violation_measure], 1)
        dists = crowding_distance(scores)
        mask_f = scores[:, -1] == 0
        mask_inf = ~mask_f
        s_f = np.sum(mask_f)
        s_inf = np.sum(mask_inf)
        if s_f < n_f:
            to_take_f = s_f
            to_take_inf = n - s_f
        elif s_inf < n_inf:
            to_take_inf = s_inf
            to_take_f = n - s_inf
        else:
            to_take_f = n_f
            to_take_inf = n_inf
            
        population_f = population[mask_f]
        fronts, ranks = FNS(population_f)
        taken_f = elitist_selection(fronts, dists[mask_f], to_take_f)
        population_inf = population[mask_inf]
        fronts, ranks = FNS(population_inf)
        taken_inf = elitist_selection(fronts, dists[mask_inf], to_take_inf)
        
        population = np.vstack([population_f[taken_f, :], population_inf[taken_inf, :]])
        scores = np.vstack([scores[mask_f, :][taken_f, :], scores[mask_inf, :][taken_inf, :]])
        dists = np.hstack([dists[mask_f][taken_f], dists[mask_inf][taken_inf]])
        fronts, ranks = FNS(population)
        
        populations.append(population.copy())
        scores_hist.append(scores.copy())
        
        if iter_ % log_interval == 0:
            print(f"Iteration {iter_}, #feasible: {to_take_f}, #infeasible: {to_take_inf}, scores: {scores.min(0)} {scores.mean(0)} {scores.max(0)}")
    print(f"Iteration {iter_}, #feasible: {to_take_f}, #infeasible: {to_take_inf}, scores: {scores.min(0)} {scores.mean(0)} {scores.max(0)}")
    return np.stack(populations, 0), np.stack(scores_hist, 0)

In [12]:
def g1_objective(x):
    return 5 * x[:, :4].sum(1) - 5 * (x[:, :4]**2).sum(1) - x[:, 4:].sum(1)

def g1_constraints(x):
    c = np.stack([
        2*x[:, 0] + 2*x[:, 1] + x[:, 9] + x[:, 10] - 10.,
        2*x[:, 0] + 2*x[:, 2] + x[:, 9] + x[:, 11] - 10.,
        2*x[:, 1] + 2*x[:, 2] + x[:, 10] + x[:, 11] - 10.,
        -8*x[:, 0] + x[:, 9],
        -8*x[:, 1] + x[:, 10],
        -8*x[:, 2] + x[:, 11],
        -2*x[:, 3] - x[:, 4] + x[:, 9],
        -2*x[:, 5] - x[:, 6] + x[:, 10],
        -2*x[:, 7] - x[:, 8] + x[:, 11],
    ], 1)
    c = np.maximum(c, 0.)
    return c
    
def g6_objective(x):
    return (x[:, 0] - 10.)**3 + (x[:, 1] - 20.)**3

def g6_constraints(x):
    c = np.stack([
        -(x[:, 0] - 5)**2 - (x[:, 1]-5)**2 + 100,
        (x[:, 0] - 6)**2 + (x[:, 1]-5)**2 - 82.81,
    ], 1)
    c = np.maximum(c, 0.)
    return c

def g2_objective(x):
    n, d = x.shape
    c = np.cos(x)
    return -np.abs(((c**4).sum(1) - 2*(c**2).prod(1)) / np.sqrt((np.arange(d) * x**2).sum(1)))

def g2_constraints(x):
    n, d = x.shape
    c = np.stack([
        0.75 - x.prod(1),
        x.sum(1) - 7.5*d
    ], 1)
    c = np.maximum(c, 0.)
    return c

In [20]:
objective = g6_objective
constraint = g6_constraints

x_min = np.array([13., 0.])
x_max = 100.
d = 2
n = 200
n_inf = int(0.2*n)
eta_c = 15.
eta_m = 20.
p_c = 0.8
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, constraint, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

Iteration 0, #feasible: 0, #infeasible: 200, scores: [-3.95769257e+03  1.00000000e+00] [1.18420999e+05 8.60700000e+01] [9.13276447e+05 2.95000000e+02]
Iteration 100, #feasible: 160, #infeasible: 40, scores: [-7967.53682068     0.        ] [-7031.69602586     7.68      ] [-6901.12295816    58.        ]
Iteration 200, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7159.77860739     7.215     ] [-6956.59705211    39.        ]
Iteration 300, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.13490705e+03  5.00500000e+00] [-6915.73693644    34.        ]
Iteration 400, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7162.5199491    10.215    ] [-6960.16369139    55.        ]
Iteration 500, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.16309807e+03  6.34000000e+00] [-6960.60778848    33.        ]
Iteration 600, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.16324556e+03  6.60000000e+00] [-6960.80077764    33.        ]
Iteration 700

In [21]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-6961.542212493958

In [23]:
objective = g1_objective
constraint = g1_constraints

x_min = 0.
x_max = np.ones(13)
x_max[9:12] = 100.
d = 13
n = 200
n_inf = int(0.2*n)
eta_c = 15.
eta_m = 20.
p_c = 0.8
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, constraint, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

Iteration 0, #feasible: 0, #infeasible: 200, scores: [-265.4934198  153.       ] [-152.51234602 1281.33      ] [ -28.25592431 2353.        ]
Iteration 100, #feasible: 160, #infeasible: 40, scores: [-299.68147942    0.        ] [-24.85504509  49.285     ] [  0.66394623 604.        ]
Iteration 200, #feasible: 160, #infeasible: 40, scores: [-300.20410713    0.        ] [-32.4914137  57.4      ] [  1.3318332 591.       ]
Iteration 300, #feasible: 160, #infeasible: 40, scores: [-299.62171451    0.        ] [-26.671748  52.635   ] [  1.80716027 543.        ]
Iteration 400, #feasible: 160, #infeasible: 40, scores: [-300.23050515    0.        ] [-33.4404438  76.525    ] [  2.17068364 732.        ]
Iteration 500, #feasible: 160, #infeasible: 40, scores: [-295.28799247    0.        ] [-34.78090381  83.18      ] [  2.28167204 670.        ]
Iteration 600, #feasible: 160, #infeasible: 40, scores: [-300.47517118    0.        ] [-34.73596888  78.575     ] [  2.51745661 722.        ]
Iteration 700, #f

In [24]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-4.286607666470444

In [25]:
objective = g2_objective
constraint = g2_constraints

x_min = 1e-4
x_max = 10.
d = 20
n = 200
n_inf = int(0.2*n)
eta_c = 15.
eta_m = 20.
p_c = 0.8
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, constraint, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

Iteration 0, #feasible: 200, #infeasible: 0, scores: [-0.18063982  0.        ] [-0.12018422  0.        ] [-0.07194513  0.        ]


/home/michal/miniconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/home/michal/miniconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in greater


Iteration 100, #feasible: 160, #infeasible: 40, scores: [-0.29351088  0.        ] [-0.1445369  6.29     ] [-3.68197501e-03  5.90000000e+01]
Iteration 200, #feasible: 160, #infeasible: 40, scores: [-0.30691528  0.        ] [-0.14974879  6.345     ] [-3.20158566e-04  6.30000000e+01]
Iteration 300, #feasible: 160, #infeasible: 40, scores: [-0.31020739  0.        ] [-0.15244975  6.825     ] [-1.42062657e-04  6.60000000e+01]
Iteration 400, #feasible: 160, #infeasible: 40, scores: [-0.3108346  0.       ] [-0.15625421  7.435     ] [-9.30157655e-05  7.30000000e+01]
Iteration 500, #feasible: 160, #infeasible: 40, scores: [-0.31144428  0.        ] [-0.15480749  8.525     ] [-4.89626098e-05  7.90000000e+01]
Iteration 600, #feasible: 160, #infeasible: 40, scores: [-0.31203993  0.        ] [-0.15140525  6.925     ] [-3.0688786e-05  6.8000000e+01]
Iteration 700, #feasible: 160, #infeasible: 40, scores: [-0.31232902  0.        ] [-0.15344715  6.7       ] [-2.87049323e-05  6.30000000e+01]
Iteration 80

In [26]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-0.28779410645687825